In [1]:
# Importar librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month
import os

In [2]:
spark = (SparkSession.builder
         .appName("SECOP_Ingesta")
         .master("spark://spark-master:7077")
         .config("spark.driver.memory", "2g")
         .config("spark.executor.memory", "2g")
         .config("spark.executor.cores", "1")
         .getOrCreate())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/14 23:01:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("WARN")
print("Spark master:", spark.sparkContext.master)
print("Spark UI:", spark.sparkContext.uiWebUrl)
print(f"Spark Version: {spark.version}")
print(f"Spark Master: {spark.sparkContext.master}")

Spark master: spark://spark-master:7077
Spark UI: http://jupyter:4040
Spark Version: 3.5.0
Spark Master: spark://spark-master:7077


In [13]:
from sodapy import Socrata
import json, os
import time

client = Socrata("www.datos.gov.co", None)

out_file = "/opt/spark-data/raw/secop_q4_2025.jsonl"
open(out_file, "w").close()

total = 0

for offset in range(0, 60000, 5000):
    print(f"Consultando offset {offset}...")
    
    results = client.get(
        "jbjy-vk9h",
        query=f"""
        SELECT *
        WHERE fecha_de_firma >= '2025-10-01T00:00:00'
        AND fecha_de_firma <= '2025-12-31T23:59:59'
        LIMIT 5000
        OFFSET {offset}
        """
    )

    print("Recibidos:", len(results))

    with open(out_file, "a", encoding="utf-8") as f:
        for row in results:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    total += len(results)
    print("Total acumulado:", total)
    
    time.sleep(1)  # evita throttling agresivo

print("TOTAL FINAL:", total)



Consultando offset 0...
Recibidos: 5000
Total acumulado: 5000
Consultando offset 5000...
Recibidos: 5000
Total acumulado: 10000
Consultando offset 10000...
Recibidos: 5000
Total acumulado: 15000
Consultando offset 15000...
Recibidos: 5000
Total acumulado: 20000
Consultando offset 20000...
Recibidos: 5000
Total acumulado: 25000
Consultando offset 25000...
Recibidos: 5000
Total acumulado: 30000
Consultando offset 30000...
Recibidos: 5000
Total acumulado: 35000
Consultando offset 35000...
Recibidos: 5000
Total acumulado: 40000
Consultando offset 40000...
Recibidos: 5000
Total acumulado: 45000
Consultando offset 45000...
Recibidos: 5000
Total acumulado: 50000
Consultando offset 50000...
Recibidos: 5000
Total acumulado: 55000
Consultando offset 55000...
Recibidos: 5000
Total acumulado: 60000
TOTAL FINAL: 60000


In [14]:
import os
size_mb = os.path.getsize("/opt/spark-data/raw/secop_q4_2025.jsonl") / 1024 / 1024
print("Tamaño archivo MB:", round(size_mb, 2))


Tamaño archivo MB: 223.34


In [15]:
json_path = "/opt/spark-data/raw/secop_q4_2025.jsonl"
df_raw = spark.read.json(json_path)

print("Total de registros:", df_raw.count())
print("Total de columnas:", len(df_raw.columns))
df_raw.show(5, truncate=False)


26/02/14 23:13:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Total de registros: 60000
Total de columnas: 87
+---------+---------------+-------------+-----------------------------+--------------+----------------+-------------------------+--------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------+--------------+----------------+-------------------+---------------+--------------------------------------------+---------------------+--------------------------------+--------------------+--------+-------+---------------+-----------+---------------+-------------------------+-----------------------+----------------------------+-------------------------------------+---------------------+------------------------+--------------------------+------------------------

In [16]:
df_raw.printSchema()

root
 |-- anno_bpin: string (nullable = true)
 |-- c_digo_bpin: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- codigo_entidad: string (nullable = true)
 |-- codigo_proveedor: string (nullable = true)
 |-- condiciones_de_entrega: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- descripcion_documentos_tipo: string (nullable = true)
 |-- destino_gasto: string (nullable = true)
 |-- dias_adicionados: string (nullable = true)
 |-- documento_proveedor: string (nullable = true)
 |-- documentos_tipo: string (nullable = true)
 |-- domicilio_representante_legal: string (nullable = true)
 |-- duraci_n_del_contrato: string (nullable = true)
 |-- el_contrato_puede_ser_prorrogado: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- es_grupo: string (nullable = true)
 |-- es_pyme: string (nullable = true)
 |-- e

In [17]:
df_raw.show()

+---------+---------------+--------------+-----------------------------+--------------+----------------+----------------------+--------------------+-----------------------+---------------------------+--------------+----------------+-------------------+---------------+-----------------------------+---------------------+--------------------------------+--------------------+--------+-------+---------------+-----------+---------------+-------------------------+--------------------+----------------------------+-------------------------------------+---------------------+------------------------+--------------------------+------------------------+------------------+----------------------------------+--------------------------+-----------+--------------------+-------------------------+----------------+-------------------------------------+---------------------------------------+------------------------------+--------------------------------+-----------+----------------+--------------------+---

In [18]:
output_path = "/opt/spark-data/processed/secop_bronze_q4_2025"

(df_raw.write
 .mode("overwrite")
 .parquet(output_path))


In [19]:
spark.read.parquet(output_path).count()


60000

26/02/14 23:50:33 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
26/02/14 23:50:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce